In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
import numpy as np
from sklearn.preprocessing import StandardScaler  

train = pd.read_csv("train_merged.csv", low_memory=False)
# pd.set_option('display.max_columns', 500)
test = pd.read_csv("test_merged.csv", low_memory=False)
# targets
act_classes = ['sd', 'b', 'sv', '%', 'fc', 'aa', 'ba', 'qy', 'x', 'ny']

In [2]:
# training processing
train_copy = train.copy()
# use only rows w most freq tags
data = train_copy[train_copy['act_tag'].isin(act_classes)]


In [3]:
# test processing
test_copy = test.copy()
# use only rows w most freq tags
test_data = test_copy[test_copy['act_tag'].isin(act_classes)]

In [4]:
# targets
train_targets = data['act_tag']
# targets
test_targets = test_data['act_tag']
y_test = np.array(test_targets)

In [5]:
# cue phrases features 
cue_df      = pd.read_csv("cue_train.csv", low_memory=False)
cue_test_df = pd.read_csv("cue_test.csv", low_memory=False)


In [6]:
# unigram features 
uni_df      = pd.read_csv("unigram_train.csv", low_memory=False)
uni_test_df = pd.read_csv("unigram_test.csv", low_memory=False)


In [7]:
# pos features 
pos_df      = pd.read_csv("pos_train.csv", low_memory=False)
pos_test_df = pd.read_csv("pos_test.csv", low_memory=False)


In [11]:
print("HEY")
combined = pd.concat([cue_df, uni_df, pos_df], axis=1, join='inner')
combined_test = pd.concat([cue_test_df, uni_test_df, pos_test_df], axis=1, join='inner')

# just the new features 
combined.to_csv('combined_train.csv')
combined_test.to_csv('combined_test.csv')

X_train = np.array(combined)
y_train = np.array(train_targets)
X_test = np.array(combined_test)

HEY


In [12]:
# standardize 
scaler = StandardScaler()  
# fit only on training data 
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
# apply same transformation to test data
X_test = scaler.transform(X_test) 

In [13]:
print ("CLF") 
clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, 
                    alpha=0.0001, early_stopping=True, 
                    solver='lbfgs', verbose=10,
                    random_state=21,tol=0.000000001)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)



CLF


In [14]:
print(confusion_matrix(y_test,y_pred))


[[1986   25  122    4    2    2    4  230   65    0]
 [   9  372  389   11    4    8    1   43   21    1]
 [ 113  232 5734   25   15   15    0   18    3    2]
 [   8  212   24  355   16    0    6   66   68    1]
 [   4    8   81    8  202    0    1   59   27    1]
 [   3   59  329    0    0   21    0    0    1    0]
 [  14   12    3    6    1    0  189  205   85    1]
 [ 227   66   10   30   40    1  178 8111 1622    2]
 [  85   73    5   63   12    0   76 2199 1638    1]
 [   0    1    0    1    1    0    0    3    0  563]]


In [15]:
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           %       0.81      0.81      0.81      2440
          aa       0.35      0.43      0.39       859
           b       0.86      0.93      0.89      6157
          ba       0.71      0.47      0.56       756
          fc       0.69      0.52      0.59       391
          ny       0.45      0.05      0.09       413
          qy       0.42      0.37      0.39       516
          sd       0.74      0.79      0.76     10287
          sv       0.46      0.39      0.43      4152
           x       0.98      0.99      0.99       569

   micro avg       0.72      0.72      0.72     26540
   macro avg       0.65      0.58      0.59     26540
weighted avg       0.71      0.72      0.71     26540



In [16]:
cm = confusion_matrix(y_test,y_pred)
cm.diagonal()/cm.sum(axis=1)


array([0.81393443, 0.4330617 , 0.93129771, 0.46957672, 0.51662404,
       0.05084746, 0.36627907, 0.78847089, 0.39450867, 0.98945518])

In [17]:
accuracy_score(y_test, y_pred)


0.7223436322532027